# Blogging and Technology Debt  (upgrading a Pelican ecosystem)

## Introduction

At long last I decided to upgrade my rather ancient laptop (Windows 11 doesn't even support the CPU!).  This meant that I had to port my archive of blog content to the new laptop, and that's when I realized I had to pay back the Technology Debt I had been storing up for about seven years or more.

I had two choices:  I could upgrade all my blogging utilities (based on Pelican) to their latest versions, and suffer the pain, or try to recreate the exact same versions and environment as on my original laptop.  I chose to upgrade, as a learning experience.

## First Steps

The first steps were to create a ```conda``` environment, based upon Python 3.11, and then install Pelican, and Jupyter.

The next step was to use GitHub Descktop to clone my blog content, and current generated HTML, on the new laptop.  So far, so good.

Most of my blog posts ae composed in a Jupyter Notebook, and I rely on a Pelican plugin to convert them to HMTL in the context of my blog.  The first glitch was the the Pelican plugin that I was using wasn't much supported any more.

>    NOT MAINTAINED>    
>    I have not used this project myself on a long time. 
>    >    
>    No issues or PRs can be created   >    
I have moved on to mkdocs and my mkdocs-jupyter plugin and I recommend to do the same


## New Jupyter plugin

There were a number of choices.  I elected to go for the package ```pelican-jupyter``` from the ```pypi.org``` (https://pypi.org/project/pelican-jupyter/).  It required the smallest about of work to maintain the meta-data used to build the blog.  Essentially, all I had to do was to rename my current collection of meta-data files to have a new extension.  The format of the meta-data file was unchanged.

One problem was that this could only be installed by PIP, but this will be a very static environment, and thus mixing conda and pip would not be a problem for environment management.

So now we have:


| Name                    | Version        |           Build   | Channel     |
|----                     | ----           | ----              | ----        |
|pelican                  | 4.9.1          |    pyhd8ed1ab_0   | conda-forge |
|pelican-jupyter          | 0.10.1         |          pypi_0   | pypi        |


## Renaming utility

The little utility I wrote to rename my meta-data files is shown below.

```
"""
rename all files in a list of file names with a new extension

inputs: the file files.txt must hold names of files to change

"""

from pathlib import Path

DIR_NAME = 'C:/Users/donrc/Documents/GitHubRepos/StaticWeb/GitHubPages/coolum001.github.io-content'

def rename_file(name:str, dir_name:str)-> None:
    """
        rename_file: change extension of a file

        Parameters:
        name: name of file
        dir_name: absolute path to file

        Side Effects:
        name has suffix changed to "nbdata"

        Returns:
        None
    """
    p = Path(Path(dir_name) / name)
    new_p = p.with_suffix('.nbdata')
    p.rename(new_p)

#end rename_file

with open('files.txt', 'r') as f:
    for line in f:
        words = line.split()
        print(f'Renaming {words[-1]}')
        rename_file(words[-1], DIR_NAME)
    #end for
#end with
```

## And now the pain

So now when I run the Pelican command to "Produce-My-Blog", Pelican is now prepared to look at my Jupyter Notebooks in the blog content directory.

Sadly, what I got was:

> ERROR: Could not process ***.ipynb  
>  | basic

Éxploring the issue, it seems that at some stage, the ```nbconvert``` utility (that Pelican plugin uses) decided to have a major redesign of the storage and access of themes (maybe to support inheritence?).  All this happened about four years ago. Part of the changes was renaming the base theme from `basic` to `base`.

The plugin is located at `anaconda3\envs\staticweb\Lib\site-packages\pelican_jupyter`.  In the `core.py` file, my version now reads (at line 87):

>     """Return the HTML from a Jupyter Notebook>
    """>
    # template_file = "basic>"
    template_file = "base"



## and more pain

nbconvert generated more pain.  At some stage, nbconvert decided to generate HTML anchor links for all headers.  So now headers look like:

    Header 1¶

The funny Paragraph symbol is called a Pilcrow.  In the generated HTML, you can click on the Pilcrow as it is a link to that header.  I think all this is to support automated Table of Contents generation.

The HTML generated looks like:}n

> ``` <h1 id="Header-1">Header 1<a class="anchor-link" href="#Header-1">¶</a></h1> ```

To my eyes, the Pilcrows look horrible and distracting, and I don't use Tables of Contents.  What to do?

I looked for some time to find a way to get Pelican configuration to feed into nbconvert configaturion to turn this feature off, but found no easy way.   Finally I gave in, and used a suggestion from stackoverflow.  The Pilcrow is inside an anchor tag with class "anchor-link", and that class is used nowhere else.

So I added a css command to reduce the font size to zero for these characters

/* supress Pilcrow */

.anchor-link{
    font-size: 0;
}

These css lines were added to the ```\pelican-themes-master\built-texts\local.css``` file. I use the built-texts theme for my blog.

## Conclusions

Only after a little bit of pain (and lots of Google-ing), I was able to get my blog-generation environment working again, with all the latest software.  So now I can relax for another seven years, till nxet time.